## check: 

###### do good images have empty annotation files, or no annotation files?
###### training config file: edit pipeline.config file , or download & edit as done in roboflow  (the 2 files are somewhat different)
###### load tensorboard; check model training progress
################################################## 
#### continue training
#### compare pred & GT boxes
#### get AP for each class
###################################################### 
###### labelmap creation: manually or through script?
###### how frequently are intermediate training files (model checkpoints) saved?
###### model hyperparameters: batch_size; no_of_steps (parity in parameter tuning b/w yolo v4 & other models) 

In [ ]:
# use of checkpoints to autosave: https://towardsdatascience.com/checkpointing-deep-learning-models-in-keras-a652570b8de6


# custom object detection using efficientDet (Tensorflow)

In [ ]:
## TRAINING
# https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html 
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md  (ZOO)

## FULL PIPELINE
# https://colab.research.google.com/drive/1sLqFKVV94wm-lglFq_0kGo2ciM0kecWD#scrollTo=fF8ysCfYKgTP&uniqifier=1
# https://towardsdatascience.com/how-to-train-a-tensorflow-2-object-detection-model-25d4da64b817
# https://blog.roboflow.com/getting-started-with-roboflow/
 
# https://gilberttanner.com/blog/tensorflow-object-detection-with-tensorflow-2-creating-a-custom-model
# https://missinglink.ai/guides/tensorflow/tensorflow-image-recognition-object-detection-api-two-quick-tutorials/

# https://medium.com/practical-deep-learning/a-complete-transfer-learning-toolchain-for-semantic-segmentation-3892d722b604


## SAMPLE NOTEBOOK IN TENSORFLOW
~/models/research/object_detection/colab_tutorials/object_detection_tutorial.ipynb
https://colab.research.google.com/drive/1K--C2yBxIDxxX-e14Cmsx1NKeuQfyHea


####  KEY STEPS  ####
## install TF & TF object detection api
## create appropriate folder structure

## get images & label them
## split into train-test
## create labelmap (.pbtxt) & TFrecords 

## set up training - with config file & other parameters

## evaluation & model saving

## inference

In [ ]:
# connect to google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/VQI/efficientDet/tensorflow')
#!ls

In [ ]:
!nvidia-smi

Mon Nov 23 10:19:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#import os
#os.getcwd()

'C:\\Users\\DAR9KOR\\Desktop\\data\\sample_datasets\\defect_detection\\2_codes\\NEU_detection_efficientdet'

# INSTALL RELEVANT LIBRARIES

In [ ]:
# tensorflow - with CUDA, CuDNN & appropriate drivers
# https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html#tf-install

In [ ]:
# tensorflow object detction
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API+ Protobuf    #### required everytime?
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/drive/My Drive/VQI/efficientDet/tensorflow/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1571721 sha256=fa8dca35fdb653caadeac4c0a7b6a00b1067247228716c0da6ae7de57e0d1cf3
  Stored in directory: /tmp/pip-ephem-wheel-cache-j3_9phag/wheels/8e/27/ca/19fe337ff390bd7a828361114a767605af5cc7b04bcb8aa323
  Created wheel for avro-python3: filename=avro_python3-1.10.0-cp36-none-any.whl size=43735 sha256=2f20ea37bf3ea8cb9b6b0767f82207e600f532f196fa70c46f69642a0bc0e5c6
  Stored in directory: /root/.cache/pip/wheels/3f/15/cd/fe4ec8b88c130393464703ee8111e2cddebdc40e1b59ea85e9
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=3068eb39ba209c12ee1b57640453c9d5dae9be1865460bb188f7768e7484d3b0
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for dill: filename=dill-0.3.1.1-cp36-none-any.whl size=78532 sha256=d4530e83d

ERROR: multiprocess 0.70.10 has requirement dill>=0.3.2, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.25.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1; python_version >= "3.0", but you'll have avro-python3 1.10.0 which is incompatible.


In [ ]:
# testing the installation, From within TensorFlow/models/research/
'''!python ./models/research/object_detection/builders/model_builder_tf2_test.py


In [ ]:
'''# pycoco tools for evaluation, if it did not get installed
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi/PythonAPI
make
cp -r pycocotools <PATH_TO_TF>/TensorFlow/models/research/

# CREATE THE WORKSPACE

In [ ]:
os.getcwd()

'/content/drive/My Drive/VQI/efficientDet/tensorflow'

In [ ]:
'''os.mkdir("workspace")
os.mkdir("workspace/training_demo")  # training folder
os.mkdir("workspace/training_demo/annotations") # list of annotations for dataset; .csv & TF .record files
os.mkdir("workspace/training_demo/exported-models")  # store exported versions of our trained model(s)

os.mkdir("workspace/training_demo/images") # images + .xml files in our dataset
os.mkdir("workspace/training_demo/images/test")  # test images + .xml
os.mkdir("workspace/training_demo/images/train") # training img + .xml

os.mkdir("workspace/training_demo/models")  # sub-folder foreach training, with training pipeline *.config file + files generated during training and evaluation of model
os.mkdir("workspace/training_demo/pre-trained-models") # downloaded pre-trained models, which is used as a starting checkpoint for training

In [ ]:
!ls ./workspace/GC10

annotations	     images		      models
exported-models      inference_images_TF2.py  pre-trained-models
exporter_main_v2.py  model_main_tf2.py


# PRE-PROCESS DATA

#### train val test splits , data augmentation , annotation file format conversion

#### code to split data into train-val-test @Astrid

In [ ]:
# from @Astrid
# alternately, script here , to split & copy: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html

#### augment data - not yet implemented - training time augmentation is set inside config file

In [ ]:
# augment data in train set - all aug
# augmnt data in val - only resizes? - check
# no augment or TTA on test data

# https://github.com/Paperspace/DataAugmentationForObjectDetection
# https://github.com/anhtuan85/Data-Augmentation-for-Object-Detection


# check how to keep track of annotations for augmented data, where image size or shape changes
# brightness, hue kind of transforms won't change the annotation locations

#### copy images + .xml annotation to appropriate folders

In [ ]:
'''import shutil

listpath = '/content/drive/My Drive/VQI/efficientDet/voc2coco/data/dataset_ids/'
in_img = '/content/drive/My Drive/VQI/efficientDet/voc2coco/data/'
in_anno = '/content/drive/My Drive/VQI/efficientDet/voc2coco/data/Annotations/'
out = '/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo/images/'

In [ ]:
'''# test
f = open(listpath+"test.txt") #"./data/dataset_ids/test.txt")
data = f.read()
test_dat = data.splitlines()

len(test_dat)

360

In [ ]:
'''# https://stackoverflow.com/questions/123198/how-do-i-copy-a-file-in-python

for i in test_dat:
    infile1 = in_img + 'test/' + i + '.jpg'
    infile2 = in_anno + i + '.xml'
    out_file = out + 'test/'            #+ i + '.jpg'
    shutil.copy2(infile1, out_file)
    shutil.copy2(infile2, out_file)
    

In [ ]:
'''# train
f = open(listpath+"train.txt") #"./data/dataset_ids/train.txt")
data = f.read()
train_dat = data.splitlines()

len(train_dat)

for i in train_dat:
    infile1 = in_img + 'train/' + i + '.jpg'
    infile2 = in_anno + i + '.xml'
    out_file = out + 'train/' 
    shutil.copy2(infile1, out_file)
    shutil.copy2(infile2, out_file)


In [ ]:
'''# val
f = open("./data/dataset_ids/val.txt")
data = f.read()
val_dat = data.splitlines()

os.mkdir('val')
len(val_dat)

for i in val_dat:
    in_file = 'C:/Users/DAR9KOR/Desktop/data/sample_datasets/defect_detection/2_codes/NEU_detection_efficientdet/voc2coco-master/data/Images/' + i + '.jpg'
    out_file = 'C:/Users/DAR9KOR/Desktop/data/sample_datasets/defect_detection/2_codes/NEU_detection_efficientdet/voc2coco-master/val/' #+ i + '.jpg'
    shutil.copy2(in_file, out_file)
    

#### create Tensorflow labelmap (classname -> integer mapping)

In [ ]:
# label_map.pbtxt in training_demo/annotations folder; without quotes
# currently done manually; can this be scripted?

'''

item {
    id: 1
    name: 'cat'
}

item {
    id: 2
    name: 'dog'
}

'''

#### create TF Records (convert .xml -> .record)

In [ ]:
'''import pandas

In [ ]:
'''os.getcwd()

'/content/drive/My Drive/VQI/efficientDet/tensorflow'

In [ ]:
'''# Create train data:
!python ./scripts/preprocessing/generate_tfrecord.py -x ./workspace/training_demo/images/train -l ./workspace/training_demo/annotations/label_map.pbtxt -o ./workspace/training_demo/annotations/train.record 

# Create test data:
!python ./scripts/preprocessing/generate_tfrecord.py -x ./workspace/training_demo/images/test -l ./workspace/training_demo/annotations/label_map.pbtxt -o ./workspace/training_demo/annotations/test.record 



In [ ]:
'''!ls ./workspace/training_demo/images/test/ | grep 'patches_12'

In [ ]:
'''!ls ./workspace/training_demo/images/train/ | grep 'patches_104'

In [ ]:
'''import cv2
cv2.imread("./workspace/training_demo/images/train/patches_104.jpg")

In [ ]:
'''# https://github.com/tensorflow/models/issues/3762
# explicitly added -i images_folder 

# Create train data:
!python ./scripts/preprocessing/generate_tfrecord.py \
-x ./workspace/training_demo/images/train \
-l ./workspace/training_demo/annotations/label_map.pbtxt \
-o ./workspace/training_demo/annotations/train.record \
-i ./workspace/training_demo/images/train

# Create test data:
!python ./scripts/preprocessing/generate_tfrecord.py \
-x ./workspace/training_demo/images/test \
-l ./workspace/training_demo/annotations/label_map.pbtxt \
-o ./workspace/training_demo/annotations/test.record \
-i ./workspace/training_demo/images/test



In [ ]:
'''# Create train data:
!python /content/drive/My\ Drive/VQI/efficientDet/tensorflow/scripts/preprocessing/generate_tfrecord.py \
-x /content/drive/My\ Drive/VQI/efficientDet/tensorflow/workspace/training_demo/images/train \
-l /content/drive/My\ Drive/VQI/efficientDet/tensorflow/workspace/training_demo/annotations/label_map.pbtxt \
-o /content/drive/My\ Drive/VQI/efficientDet/tensorflow/workspace/training_demo/annotations/train.record \
-i /content/drive/My\ Drive/VQI/efficientDet/tensorflow/workspace/training_demo/images/train

# Create test data:
!python /content/drive/My\ Drive/VQI/efficientDet/tensorflow/scripts/preprocessing/generate_tfrecord.py \
-x /content/drive/My\ Drive/VQI/efficientDet/tensorflow/workspace/training_demo/images/test \
-l /content/drive/My\ Drive/VQI/efficientDet/tensorflow/workspace/training_demo/annotations/label_map.pbtxt \
-o /content/drive/My\ Drive/VQI/efficientDet/tensorflow/workspace/training_demo/annotations/test.record \
-i /content/drive/My\ Drive/VQI/efficientDet/tensorflow/workspace/training_demo/images/test


In [ ]:
'''!ls

efficientdet_TF_training.ipynb	models	scripts  workspace


In [ ]:
'''# create csv files from xml annotation
!python ./scripts/preprocessing/xml_to_csv_DT.py

Successfully converted xml to csv.


In [ ]:
'''# create TF records from csv
# in the script, Change import tensorflow as tf to import tensorflow.compat.v1 as tf
!python ./scripts/preprocessing/generate_tfrecord_DT.py \
--csv_input=./workspace/training_demo/images/train_labels.csv \
--image_dir=./workspace/training_demo/images/train \
--output_path=./workspace/training_demo/annotations/train.record

!python ./scripts/preprocessing/generate_tfrecord_DT.py \
--csv_input=./workspace/training_demo/images/test_labels.csv \
--image_dir=./workspace/training_demo/images/test \
--output_path=./workspace/training_demo/annotations/test.record

2020-09-21 06:44:38.612420: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Successfully created the TFRecords: /content/drive/My Drive/VQI/efficientDet/tensorflow/./workspace/training_demo/annotations/train.record
2020-09-21 06:52:25.152344: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Successfully created the TFRecords: /content/drive/My Drive/VQI/efficientDet/tensorflow/./workspace/training_demo/annotations/test.record


# CONFIGURE TRAINING

### download pre-trained model weights from TF model zoo

In [ ]:
'''os.getcwd()

'/content/drive/My Drive/VQI/efficientDet/tensorflow'

In [ ]:
'''os.chdir("workspace/training_demo/pre-trained-models/")
os.getcwd()

'/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo/pre-trained-models'

In [ ]:
'''os.getcwd()
os.chdir("pre-trained-models")

In [ ]:
'''os.getcwd()

'/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo/pre-trained-models'

In [ ]:
'''##change chosen model to deploy different models available in the TF2 object detection zoo
MODELS_CONFIG = {
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
        'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 16
    }
}

#in this tutorial we implement the lightweight, smallest state of the art efficientdet model
#if you want to scale up tot larger efficientdet models you will likely need more compute!
chosen_model = 'efficientdet-d1'

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [ ]:
'''import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

## configure training pipeline

In [ ]:
'''num_steps = 40000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing. 
num_eval_steps = 500 #Perform evaluation after so many steps
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your training

In [ ]:
'''os.mkdir("../models/efficientdet_d1-round2")
#!cp ./efficientdet_d1_coco17_tpu-32/pipeline.config ../models/efficientdet_d1

In [ ]:
# edit the pipeline.config file in ../models/model_name_location_folder/pipeline.config:

# num_classes (label map?)
# batch size
# fine_tune_checkpoint, ie path to checkpoint of pretrained model
# fine_tune_checkpoint_type: detection
# use_bfloat: set to False if not training on TPU
# train-test records
# no. of steps

In [ ]:
'''# edit config file from website
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

##### should we edit the file in models/efficientdet_d1/pipeline_config; or download the config file from modelzoo & edit, as done by roboflow? 
####### the first file has y_scale: 1.0, x_scale: 1.0, height_scale: 1.0,  width_scale: 1.0 & all decimals with high precision (many digits)
####### the 2nd file has lower precision (single decimal digits) & y_scale: 10.0, x_scale: 10.0, height_scale: 5.0,  width_scale: 5.0

In [ ]:
'''#prepare
pipeline_fname = '/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo/pre-trained-models/' + base_pipeline_file
fine_tune_checkpoint = '/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo/pre-trained-models/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

test_record_fname = '/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo/annotations/test.record'
train_record_fname = '/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo/annotations/train.record'
label_map_pbtxt_fname = '/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo/annotations/label_map.pbtxt'
num_classes = get_num_classes(label_map_pbtxt_fname)

In [ ]:
'''os.getcwd()

In [ ]:
'''#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re

fine_tune_checkpoint = 'pre-trained-models/' + model_name + '/checkpoint/ckpt-0'
test_record_fname = 'annotations/test.record'
train_record_fname = 'annotations/train.record'
label_map_pbtxt_fname = 'annotations/label_map.pbtxt'
batch_size = 8
#tpu_option = false


#%cd /content/models/research/deploy
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
    
    # turn off tpu
    s = re.sub('use_bfloat16: true',
               'use_bfloat16: {}'.format('false'), s)

        
    f.write(s)



writing custom configuration file


In [ ]:
'''%cat pipeline_file.config

In [ ]:
'''os.getcwd()

'/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo/pre-trained-models'

## train

In [ ]:
'''!cp ../../../models/research/object_detection/model_main_tf2.py ../

In [ ]:
#os.chdir("../")
#os.chdir("workspace/training_demo")
os.getcwd()

'/content/drive/My Drive/VQI/efficientDet/tensorflow'

In [ ]:
'''!cp ./pre-trained-models/pipeline_file.config ./models/efficientdet_d1-round2/pipeline.config

In [ ]:
'''
# to avoid colab disconnection
press ctrl+shift+i simultaneously. This will open console and in the console paste the following code.

function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}setInterval(ClickConnect,60000)

In [ ]:
os.chdir("workspace/GC10")

In [ ]:
# run training

#!python model_main_tf2.py \
#    --pipeline_config_path=models/efficientdet_d1-round2/pipeline.config \
#    --model_dir=models/efficientdet_d1-round2 \
#    --alsologtostderr \
#    --num_train_steps=40000 \
#    --sample_1_of_n_eval_examples=1 \
#    --num_eval_steps=500

!python model_main_tf2.py \
    --pipeline_config_path=models/effd1_highdec_cus2/pipeline_effd1_highdec_custom2.config \
    --model_dir=models/effd1_highdec_cus2 \
    --num_train_steps=32000 \
    --num_eval_steps=500 \
    --alsologtostderr

# code failed twice (after the WARNING: gradients do not exist) for batch_size = 16; resetting to 8 & increasing steps tp 32000

2020-11-18 10:50:20.504382: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-18 10:50:25.835824: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-18 10:50:25.903208: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-18 10:50:25.903800: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-11-18 10:50:25.903870: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-18 10:50:26.133507: I tensorflow/stream_executor/platform/default

## model evaluation

In [ ]:
# From the models directory
# https://towardsdatascience.com/custom-object-detection-using-tensorflow-from-scratch-e61da2e10087

# eval.py code is absent in TF2
'''
!python research/object_detection/eval.py \
    --logtostderr \
    --pipeline_config_path=ssd_mobilenet_v2_coco.config \
    --checkpoint_dir=train \
    --eval_dir=eval


In [ ]:
os.getcwd()

'/content/drive/My Drive/VQI/efficientDet/tensorflow'

In [ ]:
#os.chdir("workspace/training_demo/")

In [ ]:
# https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html
# this creates a new folder "eval" inside current dir
!python model_main_tf2.py \
    --model_dir=models/effd1_highdec_cus2 \
    --pipeline_config_path=models/effd1_highdec_cus2/pipeline_effd1_highdec_custom2.config \
    --checkpoint_dir=models/effd1_highdec_cus2 \
    --alsologtostderr   ## needed to add this option to generate log file


2020-11-19 06:17:07.078236: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
W1119 06:17:10.388647 140491992930176 model_lib_v2.py:925] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I1119 06:17:10.388919 140491992930176 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1119 06:17:10.389012 140491992930176 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1119 06:17:10.389095 140491992930176 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1119 06:17:10.389216 140491992930176 model_lib_v2.py:940] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
2020-11-19 06:17:10.414674: I tensorflow/stream_executor/pl

In [ ]:
# restarting training

!python model_main_tf2.py \
    --pipeline_config_path=models/effd1_highdec_cus2/pipeline_effd1_highdec_custom2.config \
    --model_dir=models/effd1_highdec_cus2 \
    --num_train_steps=32000 \
    --num_eval_steps=500 \
    --alsologtostderr

2020-11-19 06:34:22.759185: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-19 06:34:22.759185: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-19 06:34:24.715389: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-19 06:34:24.732043: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-19 06:34:24.732591: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-11-19 06:34:24.732648: I tensorflow/stream_executor/pl

In [ ]:
os.getcwd()

'/content/drive/My Drive/VQI/efficientDet/tensorflow'

In [ ]:
os.chdir("workspace/GC10/")

In [ ]:
# eval on extended training - round 1
# this creates a new folder "eval" inside current dir

!python model_main_tf2.py \
    --model_dir=models/effd1_highdec_cus2 \
    --pipeline_config_path=models/effd1_highdec_cus2/pipeline_effd1_highdec_custom2.config \
    --checkpoint_dir=models/effd1_highdec_cus2 \
    --alsologtostderr   ## needed to add this option to generate log file

2020-11-19 11:43:39.739119: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
W1119 11:43:44.681133 139988893415296 model_lib_v2.py:925] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I1119 11:43:44.681362 139988893415296 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1119 11:43:44.681458 139988893415296 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1119 11:43:44.681545 139988893415296 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1119 11:43:44.681673 139988893415296 model_lib_v2.py:940] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
2020-11-19 11:43:44.705853: I tensorflow/stream_executor/pl

In [ ]:
## restart training - round 2
# restarting training

!python model_main_tf2.py \
    --pipeline_config_path=models/effd1_highdec_cus2/pipeline_effd1_highdec_custom2.config \
    --model_dir=models/effd1_highdec_cus2 \
    --num_train_steps=32000 \
    --num_eval_steps=500 \
    --alsologtostderr


In [ ]:
## eval on extended training round 2

!python model_main_tf2.py \
    --model_dir=models/effd1_highdec_cus2 \
    --pipeline_config_path=models/effd1_highdec_cus2/pipeline_effd1_highdec_custom2.config \
    --checkpoint_dir=models/effd1_highdec_cus2 \
    --alsologtostderr   ## needed to add this option to generate log file

## save/ export trained model

In [ ]:
#os.getcwd()

'/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo'

In [ ]:
os.getcwd()

'/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/GC10'

In [ ]:
#!cp ../../models/research/object_detection/exporter_main_v2.py ./

In [ ]:
# creates a new directory "efficientdet_d1" inside the pre-existing dir "exported-models"
!python exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path ./models/effd1_highdec_cus2/pipeline_effd1_highdec_custom2.config \
    --trained_checkpoint_dir ./models/effd1_highdec_cus2 \
    --output_directory ./exported-models/effd1_highdec_cus2


2020-11-19 11:56:47.754806: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-19 11:56:49.876969: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-19 11:56:49.895582: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-19 11:56:49.896195: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-11-19 11:56:49.896236: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-19 11:56:49.902425: I tensorflow/stream_executor/pl

# INFERENCE

In [ ]:
import io
import os
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#n_channel = 1
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  ## convert to RGB if image is grayscale
  rgbimg = Image.new("RGB", image.size)
  rgbimg.paste(image)
  return np.array(rgbimg.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)
 
  #return np.array(image.getdata()).reshape(
  #    (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
os.chdir("/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/GC10/")

#### load model & labelmap : the 2 methods primarily vary in how the model is loaded for inferencing
##### method 1 uses a .pb file
##### method 2 uses checkpoint file
##### inferencing results change based on which checkpoint is used


In [ ]:
'''
# Roboflow, method 1: https://colab.research.google.com/drive/1sLqFKVV94wm-lglFq_0kGo2ciM0kecWD#scrollTo=YnSEZIzl4M10&uniqifier=1

### load model
#recover our saved model
pipeline_config = "./models/efficientdet_d1/pipeline.config"  #pipeline_file

#generally you want to put the last ckpt from training in here
model_dir = '/content/training/ckpt-11'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)


# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join('/content/training/ckpt-19'))


def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)


###########################################################################
## map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [ ]:
# load model, method 2
# https://colab.research.google.com/github/TannerGilbert/Tensorflow-Object-Detection-API-Train-Model/blob/master/Tensorflow_2_Object_Detection_Train_model.ipynb#scrollTo=EEX-m3P1yp4y

tf.keras.backend.clear_session()
model = tf.saved_model.load(f'./exported-models/effd1_highdec_cus2/saved_model')

In [ ]:
# label map
labelmap_path = './annotations/label_map.pbtxt'  #'/content/labelmap.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)

#### run inference

In [ ]:
'''
# roboflow, method 1
#run detector on test image
#it takes a little longer on the first run and then runs at normal speed. 
import random

TEST_IMAGE_PATHS = glob.glob('/content/test/test/*.jpg')
image_path = random.choice(TEST_IMAGE_PATHS)
image_np = load_image_into_numpy_array(image_path)

# Things to try:
# Flip horizontally
# image_np = np.fliplr(image_np).copy()

# Convert image to grayscale
# image_np = np.tile(
#     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.5,
      agnostic_mode=False,
)

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.show()

In [ ]:
# method 2
# https://colab.research.google.com/github/TannerGilbert/Tensorflow-Object-Detection-API-Train-Model/blob/master/Tensorflow_2_Object_Detection_Train_model.ipynb#scrollTo=EEX-m3P1yp4y

def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
import glob
for image_path in glob.glob('./images/val/*.jpg'):
  print("")
  print(image_path)
  img = image_path.split("/")[-1]
  outfile = "./images/val_out/" + img
  image_np = load_image_into_numpy_array(image_path)
  output_dict = run_inference_for_single_image(model, image_np)
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)
  #display(Image.fromarray(image_np))
  plt.figure(figsize=(5,5))
  plt.imshow(image_np) #_with_detections)
  #plt.show()
  plt.savefig(outfile)
  plt.close()


./images/val/img_02_425506300_00018.jpg

./images/val/img_02_425503100_00017.jpg

./images/val/img_02_425505500_00018.jpg

./images/val/img_02_425506400_00018.jpg

./images/val/img_03_3403394000_01158.jpg

./images/val/img_03_3403392100_00870.jpg

./images/val/img_03_3403404800_00998.jpg

./images/val/img_03_3403400200_00567.jpg

./images/val/img_03_3403400500_00340.jpg

./images/val/img_03_425364500_00001.jpg

./images/val/img_03_3403402900_00962.jpg

./images/val/img_03_425501800_00018.jpg

./images/val/img_03_425501900_00017.jpg

./images/val/img_03_3403405900_00857.jpg

./images/val/img_03_3403401700_00908.jpg

./images/val/img_03_425505100_00017.jpg

./images/val/img_03_425505900_00018.jpg

./images/val/img_03_425507200_00018.jpg

./images/val/img_03_425640000_00634.jpg

./images/val/img_03_425506400_00018.jpg

./images/val/img_03_425639800_00874.jpg

./images/val/img_03_425638500_00716.jpg

./images/val/img_06_3403399100_01155.jpg

./images/val/img_06_3403391400_01266.jpg

./ima

####  inference speed on GPU (K80), method 2 

In [ ]:
import glob
import time

for image_path in glob.glob('./images/sample_val_infer/*.jpg'):
  print("")
  print(image_path)
  start_time=time.time()
  image_np = load_image_into_numpy_array(image_path)
  output_dict = run_inference_for_single_image(model, image_np)
  print(" Inference time: %s seconds " % (time.time() - start_time)) 
  #viz_utils.visualize_boxes_and_labels_on_image_array(
  #    image_np,
  #    output_dict['detection_boxes'],
  #    output_dict['detection_classes'],
  #    output_dict['detection_scores'],
  #    category_index,
  #    instance_masks=output_dict.get('detection_masks_reframed', None),
  #    use_normalized_coordinates=True,
  #    line_thickness=8)
  #display(Image.fromarray(image_np))

##### inference speed on CPU, method 2

In [ ]:
import glob
import time

for image_path in glob.glob('./images/sample_val_infer/*.jpg'):
  print("")
  print(image_path)
  start_time=time.time()
  image_np = load_image_into_numpy_array(image_path)
  output_dict = run_inference_for_single_image(model, image_np)
  print(" Inference time: %s seconds " % (time.time() - start_time))

#### inferencing speed with method 1 (roboflow method) - slightly faster

In [ ]:
os.getcwd()

'/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo'

In [ ]:
### LOAD MODEL & LABELMAP
# Roboflow method: https://colab.research.google.com/drive/1sLqFKVV94wm-lglFq_0kGo2ciM0kecWD#scrollTo=YnSEZIzl4M10&uniqifier=1

### load model
#recover our saved model
pipeline_config = "./exported-models/effd1_highdec_cus2/pipeline.config"  #pipeline_file

#generally you want to put the last ckpt from training in here
model_dir = './models/effd1_highdec_cus2/ckpt-33'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)


# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join('./models/effd1_highdec_cus2/ckpt-33'))


def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)


###########################################################################
## map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)


########################################################################

# INFER

# roboflow
#run detector on test image
#it takes a little longer on the first run and then runs at normal speed. 
import random

TEST_IMAGE_PATHS = glob.glob('./images/val/*.jpg')
image_path = random.choice(TEST_IMAGE_PATHS)
print(image_path)
image_np = load_image_into_numpy_array(image_path)

# Things to try:
# Flip horizontally
# image_np = np.fliplr(image_np).copy()

# Convert image to grayscale
# image_np = np.tile(
#     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.5,
      agnostic_mode=False,
)

plt.figure(figsize=(3,3))  #12,16
plt.imshow(image_np_with_detections)
plt.show()

./images/val/img_05_425622400_00001.jpg


ValueError: ignored

In [ ]:
import glob
import time

for image_path in glob.glob('./images/sample_val_infer/*.jpg'):
  print("")
  print(image_path)
  start_time=time.time()
  image_np = load_image_into_numpy_array(image_path)
  #output_dict = run_inference_for_single_image(model, image_np)
  input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
  detections, predictions_dict, shapes = detect_fn(input_tensor)
  
  label_id_offset = 1
  #image_np_with_detections = image_np.copy()
  print(" Inference time: %s seconds " % (time.time() - start_time))

In [ ]:
for image_path in glob.glob('./images/sample_val_infer/*.jpg'):
  print("")
  print(image_path)

  #start_time=time.time()
  image_np = load_image_into_numpy_array(image_path)
  #output_dict = run_inference_for_single_image(model, image_np)
  input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
  detections, predictions_dict, shapes = detect_fn(input_tensor)
  
  label_id_offset = 1
  image_np_with_detections = image_np.copy()
  #print(" Inference time: %s seconds " % (time.time() - start_time))

  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.5,
      agnostic_mode=False,
      )
  
  plt.figure(figsize=(3,3))  #12,16
  plt.imshow(image_np_with_detections)
  plt.show()

##### using a different checkpoint

In [ ]:
### LOAD MODEL & LABELMAP
# Roboflow method: https://colab.research.google.com/drive/1sLqFKVV94wm-lglFq_0kGo2ciM0kecWD#scrollTo=YnSEZIzl4M10&uniqifier=1

### load model
#recover our saved model
pipeline_config = "./exported-models/efficientdet_d1-round2/pipeline.config"  #pipeline_file

#generally you want to put the last ckpt from training in here
model_dir = './models/efficientdet_d1-round2/ckpt-18'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)


# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join('./models/efficientdet_d1-round2/ckpt-18'))


def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)


In [ ]:
###########################################################################
## map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)


########################################################################
for image_path in glob.glob('./images/sample_val_infer/*.jpg'):
  print("")
  print(image_path)

  start_time=time.time()
  image_np = load_image_into_numpy_array(image_path)
  #output_dict = run_inference_for_single_image(model, image_np)
  input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
  detections, predictions_dict, shapes = detect_fn(input_tensor)
  
  label_id_offset = 1
  image_np_with_detections = image_np.copy()
  print(" Inference time: %s seconds " % (time.time() - start_time))

  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.5,
      agnostic_mode=False,
      )
  
  plt.figure(figsize=(3,3))  #12,16
  plt.imshow(image_np_with_detections)
  plt.show()


In [ ]:
##################################################################################

#### only inference speed

In [ ]:
#https://github.com/google/automl/tree/master/efficientdet

start1 = time.time()
!python /content/detect/automl/efficientdet/model_inspect.py \
   --runmode=infer \
   --model_name={MODEL} \
   --ckpt_path={ckpt_path} \
   --input_image={img_path} \
   --output_image_dir={img_out_dir}
end1 = time.time()

In [ ]:
os.getcwd()

'/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo'

In [ ]:
current_path = os.getcwd()
current_path

'/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo'

In [ ]:
MODEL = 'efficientdet_d1-round2/checkpoint'
ckpt_path = current_path + '/exported-models/' + MODEL

In [ ]:
img_path = current_path + '/images/sample_val_infer/patches_299.jpg'

img_out_dir = os.path.join(os.getcwd(), 'img_out')
if not os.path.exists(img_out_dir):
  os.mkdir(img_out_dir)

In [ ]:
os.chdir("/content/drive/My Drive/VQI/efficientDet/tensorflow/workspace/training_demo/infer_speed")

In [ ]:
import sys

if "efficientdet" not in os.getcwd():
  !git clone --depth 1 https://github.com/google/automl
  #!git clone --depth 4 https://github.com/google/automl
  os.chdir('automl/efficientdet')
  sys.path.append('.')
else:
  !git pull

In [ ]:
MODEL = 'efficientdet_d1'

In [ ]:
!python /content/drive/My\ Drive/VQI/efficientDet/tensorflow/workspace/training_demo/infer_speed/automl/efficientdet/model_inspect.py \
   --runmode=infer \
   --model_name={MODEL} \
   --ckpt_path={ckpt_path} \
   --input_image={img_path} \
   --output_image_dir={img_out_dir}